In [1]:
import feedparser
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3")
conlt = engine.connect()

clean_path = "../data/clean/"
raw_path = "../data/raw/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
pdf_path = "../PDF/"

pd.set_option('display.max_colwidth', 255)
pd.set_option('display.max_rows',None)
url = "https://feeds.feedburner.com/Setorth-form45-en"

today = date.today()
year = 2022
mmdd_str = today.strftime('%m%d')
mmdd_str

'0518'

In [2]:
#today = date(2022, 5, 13)
mmdd_str = today.strftime('%m%d')
mmdd_str

'0518'

In [3]:
rss_source = feedparser.parse(url)
f45_number = len(rss_source.entries)
print("Number of F45: ", f45_number)

Number of F45:  14


In [4]:
f45_items = []

for x in range(f45_number):
    f45_content = rss_source.entries[x]
    f45_item = {}
    
    print("\n----------------------------------\n")
    
    print("F45: " + str(x))
    title_ary = f45_content.title.partition(' ')
    f45_item['name'] = title_ary[0].strip() 
    print("Title: ", f45_item['name'])  
    f45_item['year'] = year
    print("Year: ", f45_item['year'])      
    qtr_ary = title_ary[2].partition(' (F45)')
    f45_item['quarter'] = qtr_ary[0][-1]    
    print("Quarter: ", f45_item['quarter'])    
    f45_item['link'] = f45_content.link
    print("Link: ", f45_item['link'])
    f45_item['published'] = f45_content.published
    print("Published: ", f45_item['published'])  
    f45_items.append(f45_item)


----------------------------------

F45: 0
Title:  COMAN
Year:  2022
Quarter:  1
Link:  https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16528296599270
Published:  Wed, 18 May 2022 17:49:00 +0700

----------------------------------

F45: 1
Title:  AQUA
Year:  2022
Quarter:  1
Link:  https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16528296579690
Published:  Wed, 18 May 2022 09:13:13 +0700

----------------------------------

F45: 2
Title:  FTI
Year:  2022
Quarter:  1
Link:  https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16528296579210
Published:  Wed, 18 May 2022 09:06:16 +0700

----------------------------------

F45: 3
Title:  FTI
Year:  2022
Quarter:  y
Link:  https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16528296579180
Published:  Wed, 18 May 2022 09:05:59 +0700

----------------------------------

F45: 4
Title:  FTI
Year:  2022
Quarter:  1
Link:  https://classic.set.or.th/

In [5]:
df = pd.DataFrame(f45_items)
df[['name','year','quarter','published']]

,name,year,quarter,published
0,COMAN,2022,1,"Wed, 18 May 2022 17:49:00 +0700"
1,AQUA,2022,1,"Wed, 18 May 2022 09:13:13 +0700"
2,FTI,2022,1,"Wed, 18 May 2022 09:06:16 +0700"
3,FTI,2022,y,"Wed, 18 May 2022 09:05:59 +0700"
4,FTI,2022,1,"Wed, 18 May 2022 08:50:29 +0700"
5,CEN,2022,1,"Wed, 18 May 2022 08:43:12 +0700"
6,BYD,2022,1,"Wed, 18 May 2022 08:39:17 +0700"
7,KKC,2022,1,"Wed, 18 May 2022 08:38:04 +0700"
8,NCL,2022,1,"Wed, 18 May 2022 08:26:32 +0700"
9,TSR,2022,1,"Wed, 18 May 2022 08:15:08 +0700"


In [6]:
df.dtypes

name         object
year          int64
quarter      object
link         object
published    object
dtype: object

In [9]:
df.loc[(df.quarter == 'y') ,['year','quarter']] = ['2021','4']
df.groupby(['year','quarter']).count()

,,name,link,published
year,quarter,,,
2022,1,12,12,12
2021,4,2,2,2


In [10]:
df.quarter = df.quarter.astype(int)
df.groupby(['year','quarter']).count()

,,name,link,published
year,quarter,,,
2022,1,12,12,12
2021,4,2,2,2


In [11]:
df.shape

(14, 5)

In [12]:
### First equals to latest published pdf file
df = df.drop_duplicates(subset='name', keep='first')
df.shape

(11, 5)

In [13]:
file_name = 'F45-RAW-' + mmdd_str + '.csv'
raw_file = raw_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df[['name','year','quarter','published']].to_csv(output_file, header=True, index=False, sep=',')
df[['name','year','quarter','published']].to_csv(box_file,    header=True, index=False, sep=',')
df[['name','year','quarter','published']].to_csv(one_file,    header=True, index=False, sep=',')
df[['name','year','quarter','published']].to_csv(raw_file,    header=True, index=False, sep=',')

In [14]:
sql = '''
SELECT *
FROM exempts
ORDER BY name'''
df_exempts = pd.read_sql(sql, conlt)
df_exempts.shape[0]

405

In [15]:
df_merge = pd.merge(df, df_exempts, on='name', how='outer', indicator=True)
df_merge.shape

(409, 7)

### Tickers that won't be input

In [16]:
in_exempts = df_merge.loc[
    df_merge['_merge'] == 'both',
    ['name','year','quarter','published','link']
    
]
in_exempts.year = in_exempts.year.astype(int)
in_exempts.quarter = in_exempts.quarter.astype(int)
in_exempts.sort_values(by=['published'],ascending=[False])

,name,year,quarter,published,link
0,COMAN,2022,1,"Wed, 18 May 2022 17:49:00 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16528296599270
1,AQUA,2022,1,"Wed, 18 May 2022 09:13:13 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16528296579690
3,CEN,2022,1,"Wed, 18 May 2022 08:43:12 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16528296577880
5,KKC,2022,1,"Wed, 18 May 2022 08:38:04 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16528296577540
6,NCL,2022,1,"Wed, 18 May 2022 08:26:32 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16528296576490
7,TSR,2022,1,"Wed, 18 May 2022 08:15:08 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16528296575700
10,LHSC,2022,1,"Wed, 18 May 2022 07:02:22 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16528296573420


In [17]:
in_exempts.sort_values(by=['published'],ascending=[False]).shape[0]

7

In [18]:
df_out = df_merge.loc[
    df_merge['_merge'] == 'left_only',
    ['name','year','quarter','published','link']
]
df_out.year = df_out.year.astype(int)
df_out.quarter = df_out.quarter.astype(int)
df_out.sort_values(by=['published'],ascending=[False])

,name,year,quarter,published,link
2,FTI,2022,1,"Wed, 18 May 2022 09:06:16 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16528296579210
4,BYD,2022,1,"Wed, 18 May 2022 08:39:17 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16528296577640
8,JCK,2022,1,"Wed, 18 May 2022 07:56:17 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16528296574630
9,DIMET,2022,1,"Wed, 18 May 2022 07:49:38 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16528296573720


In [19]:
#df_out = df_out.drop(df_out.index[df_out['name'] == "SCC"])
df_out.shape[0]

4

In [20]:
sql = '''
SELECT *
FROM tickers
ORDER BY name'''
df_tickers = pd.read_sql(sql, conlt)
df_tickers.shape

(403, 9)

In [21]:
df_merge2 = pd.merge(df_out, df_tickers, on='name', how='outer', indicator=True)
df_merge2.shape

(407, 14)

### There are no ticker records

In [22]:
df_merge2.loc[
    df_merge2['_merge'] == 'left_only',
    ['name','year','quarter','published','link']
]

,name,year,quarter,published,link
0,FTI,2022.0,1.0,"Wed, 18 May 2022 09:06:16 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16528296579210
1,BYD,2022.0,1.0,"Wed, 18 May 2022 08:39:17 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16528296577640
2,JCK,2022.0,1.0,"Wed, 18 May 2022 07:56:17 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16528296574630
3,DIMET,2022.0,1.0,"Wed, 18 May 2022 07:49:38 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16528296573720


In [23]:
df_merge2.loc[
    df_merge2['_merge'] == 'left_only',
    ['name','year','quarter','published','link','id','market']
].shape

(4, 7)

### There are ticker records

In [24]:
df_out2 = df_merge2.loc[
    df_merge2['_merge'] == 'both',
    ['name','year','quarter','published','link','id','market']
]
df_out2

,name,year,quarter,published,link,id,market


In [25]:
df_out2 = df_out2[df_out2.year.notnull()]
df_out2.shape

(0, 7)

In [26]:
df_out2['year'] = df_out2['year'].astype(int)
df_out2['quarter'] = df_out2['quarter'].astype(int)
df_out2.shape

(0, 7)

In [27]:
file_name = 'F45-CLEAN-' + mmdd_str + '.csv'
clean_file = clean_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_out2[['name','year','quarter','published','market']].sort_values(['published'],ascending=[False]).to_csv(output_file, header=True, index=False, sep=',')
df_out2[['name','year','quarter','published','market']].sort_values(['published'],ascending=[False]).to_csv(clean_file, header=True, index=False, sep=',')
df_out2[['name','year','quarter','published','market']].sort_values(['published'],ascending=[False]).to_csv(box_file, header=True, index=False, sep=',')
df_out2[['name','year','quarter','published','market']].sort_values(['published'],ascending=[False]).to_csv(one_file, header=True, index=False, sep=',')

In [28]:
sql = '''
SELECT * 
FROM epss
WHERE year = 2022'''
df_epss = pd.read_sql(sql, conlt)
df_epss.shape

(218, 14)

In [59]:
df_merge3 = pd.merge(df_out2, df_epss, on=['name','year','quarter'], how='outer', indicator=True)
df_merge3.shape

(218, 19)

### Already input, display profit amt & eps to check with new F45

In [60]:
df_merge3[df_merge3['_merge'] == 'both']

,name,year,quarter,published,link,id_x,market,id_y,q_amt,y_amt,aq_amt,ay_amt,q_eps,y_eps,aq_eps,ay_eps,ticker_id,publish_date,_merge
3,ROJNA,2022,1,"Tue, 17 May 2022 17:51:58 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16527404246480,404.0,sSET,21615.0,825096.0,293107.0,825096.0,293107.0,0.4080,0.1450,0.4080,0.1450,404.0,2022-05-17,both
6,STEC,2022,1,"Tue, 17 May 2022 17:04:53 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16527404233840,484.0,SET100,21616.0,231740.0,1097487.0,231740.0,1097487.0,0.1500,0.1300,0.1500,0.1300,484.0,2022-05-17,both
12,LALIN,2022,1,"Tue, 17 May 2022 08:53:36 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16527404191790,263.0,sSET,21613.0,327606.0,319712.0,327606.0,319712.0,0.3500,0.3500,0.3500,0.3500,263.0,2022-05-17,both
13,SENA,2022,1,"Tue, 17 May 2022 08:38:32 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16527404190380,437.0,sSET,21577.0,336344.0,229318.0,336344.0,229318.0,0.2340,0.1607,0.2340,0.1607,437.0,2022-05-13,both
14,VIBHA,2022,1,"Tue, 17 May 2022 08:28:58 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16527404189330,610.0,SETWB,21609.0,454710.0,191898.0,454710.0,191898.0,0.0335,0.0141,0.0335,0.0141,610.0,2022-05-13,both
15,MCS,2022,1,"Tue, 17 May 2022 08:17:43 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16527404187130,293.0,sSET,21614.0,262711.0,233571.0,262711.0,233571.0,0.5500,0.4900,0.5500,0.4900,293.0,2022-05-17,both
16,FORTH,2022,1,"Tue, 17 May 2022 07:43:10 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16527404182240,176.0,SET,21612.0,236559.0,184217.0,236559.0,184217.0,0.2500,0.2000,0.2500,0.2000,176.0,2022-05-17,both


In [61]:
df_merge3[df_merge3['_merge'] == 'both'].shape

(7, 19)

In [62]:
df_inp2 = df_merge3[df_merge3['_merge'] == 'left_only']
df_inp3 = df_inp2.copy()
df_inp3.shape

(15, 19)

In [63]:
df_inp3['year'] = df_inp3['year'].astype(str)
df_inp3['quarter'] = df_inp3['quarter'].astype(str)
final = df_inp3.sort_values('name',ascending=True)
final_str = final.name+' '+final.year+' '+final.quarter+' '+final.link
final_str

8         AH 2022 1 https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16527404208610
11    CENTEL 2022 1 https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16527404192730
1         CK 2022 1 https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16527404259700
9        HTC 2022 1 https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16527404208550
17       IMH 2022 1 https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16527404180640
21       JWD 2022 1 https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16527404175670
7        KEX 2022 1 https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16527404231930
18     PCSGH 2022 1 https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16527404180130
4         RS 2022 1 https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16527404237650
0       SI